# Exemplo: Pressões de vapor e entalpias de vaporização de alquil formamidas

## Introdução

O problema aborda o cálculo da pressão de vapor de substâncias puras em função da temperatura, dentre estas substâncias está N-benzil-formamida, que é substância na figura 1.


**Figura 1: N-butil-formamida. Fonte: [1]**

<img src="FotoJet.png" alt="drawing" width="150"/>



São apresentados alguns dados experimentais iniciais para esta reação na Tabela 1.

**Tabela 1. Dados experimentais. Fonte: [1].**

Temperatura / K | Pressão de vapor (Pa) | Incertezas para pressão de vapor
:--------------:|:---------------------:|:-------------------------------:
        297.1   |         2.93          |        0.08 
        298.2   |         3.21          |        0.09
        299.3   |         3.49          |        0.09
        301.2   |         4.22          |        0.11
        304.2   |         5.60          |        0.17
        307.2   |         7.31          |        0.21
        310.2   |         9.12          |        0.25
        314.1   |         13.07         |        0.35
        316.2   |         14.98         |        0.40
        317.8   |         17.63         |        0.47
        318.2   |         18.02         |        0.48
        320.2   |         22.08         |        0.58
        323.1   |         26.95         |        0.70
        326.2   |         34.61         |        0.89
        329.1   |         40.93         |        0.105
          
O modelo para esta reação é dado por [1]:

$y = \exp\left(A-(\frac{B}{T+C}\right))$ (1),

em que $y$ corresponde a pressão de vapor da substância, $T$ é a temperatura, e por fim $A$, $B$ e $C$ são os parâmetros a serem estimados.

Então, o problema de otimização a ser resolvido envolve o uso da função objetivo de mínimos quadrados ponderados pelo inverso da variância, conforme [2]:

$\min_{ko, E} \sum_{i=1}^{NE} \left(\frac{y^{exp}_i-y_i(k_o,E)}{u^2_{y_i}}\right)$ (2),

sujeito à (1).

Para resolução deste problema no MT_PEU serão usados os símbolos:

* Símbolos das grandezas independentes (tempo e temperatura): T
* Símbolos das grandezas dependentes (pressão de vapor): y
* Símbolos dos parâmetros: A, B e C

## Importação de pacotes

importação de bibliotecas (pacotes) necessárias para execução do código.

* **MT_PEU**: biblioteca que contém as principais funcionalidades da ferramenta

    * Importar a classe **EstimacaoNaoLinear**, que será utilizada neste exemplo de estimação não linear
    
* **numpy**: biblioteca para computação científica -> será necessária somente a função exp (exponencial)

    * será necessária somente a função **exp** (exponencial) para construção do modelo

In [7]:
from MT_PEU import EstimacaoNaoLinear
from numpy import exp

## Criação de modelo

O modelo (1) representa o comportamento da grandeza dependente, no qual os parâmetros $A$, $B$ e $C$ serão estimados.

Este modelo é definido, então, na forma de uma subrotina, **def**, do python e representado por:

In [8]:
def Modelo(param,x,args):


    T = x[0]
    A, B, C = param[0], param[1],  param[2]
    
    
    return exp(A-(B/(T+C))) # cálculo de Pvp - vetorizado

## Inicialização de classe

Para executar a estimação, o primeiro passo é configurar a classe **EstimacaoNaoLinear**, inicialmente, incluindo informações básicas:

* O modelo,
* Os símbolos para as grandeza independentes *(x)*; 
* Os símbolos para as grandezas dependentes *(y)*; 
* Os símbolos para os parâmetros *(param)*;
* O nome do projeto, o nome da pasta onde serão gerados os resultados

In [9]:


Estimation = EstimacaoNaoLinear(Modelo, simbolos_x=[r'T'], simbolos_y=[r'y'], 
                            simbolos_param=['A','B','C'],  Folder='Exemplo5' )

## Inclusão de dados

Os dados experimentais disponibilizados em [2] da variável dependente (y) e da variáveis independentes (T), são apresentados abaixo na forma de listas:

In [10]:
#Pressão de vapor
y = [2.93,3.21,3.49,4.22,5.60,7.31,9.12,13.07,14.98,17.63,18.02,22.08,26.95,34.61,
       40.93,50.17,63.36,78.93,93.65,115.11,140.27,171.89,208.00]

#Temperatura
temperatura = [297.1,298.2,299.3,301.2,304.2,307.2,310.2,314.1,316.2,317.8,318.2,320.2,
               323.1,326.2,329.1,331.2,334.2,337.1,340.2,343.2,346.2,349.1,352.2]

Como entrada **obrigatória**, a plataforma MT_PEU necessita da **incerteza dos dados experimentais** (ux1, ux2, uy1).
Neste exemplo, foram adotados os valores das incertezas presentes em [2].

In [11]:
uxtemperatura = [0.1]*23
uy = [0.08,0.09,0.09,0.11,0.17,0.21,0.25,0.35,0.40,0.47,0.48,0.58,
      0.70,0.89,1.05,1.28,1.61,2.00,2.37,2.90,3.53,4.32,5.23]

 **Inclusão de dados experimentais na estimação:**

O método *setDados* é utilizado para incluir os dados para as grandezas dependentes e independentes. Sintaxe:

* identificação da grandeza, se independente ou dependente: 0 ou 1 (respectivamente)
* os dados experimentais e suas incertezas inseridos em sequência na forma de tuplas.

In [12]:
Estimation.setDados(0,(temperatura,uxtemperatura))
Estimation.setDados(1,(y,uy))

Define que os dados experimentais previamente inseridos serão utilizados como um conjunto de dados para o qual os 
parâmetros serão estimados:

In [13]:
Estimation.setConjunto(tipo='estimacao')

## Otimização

Neste exemplo, o usuário tem a opção de escolha do algoritmo a ser utilizado na otimização. 
Disponiveis: 'Nelder-Mead', 'Powell', 'BFGS', 'L-BFGS-B', 'CG'. 
Caso opte por não escolher, será utilizado o algoritmo default: Nelder-Mead, com estimativa inicial em [1, 1.5, 0.009].

#TODO: atualizar

In [14]:
Estimation.optimize(initial_estimative = [1, 1.5, 0.009])


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



CasADi - 2019-11-18 11:43:41 WARNING("S:nlp_f failed: Inf detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2019-11-18 11:43:41 WARNING("S:nlp_f failed: Inf detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]


## Incerteza


Neste exemplo é possível escolher o método útilizado para avaliar a incerteza 
Métodos disponíveis: 2InvHessiana, Geral, SensibilidadeModelo. 
Por definição o preenchimeto da região de verossimilhança é 'True', caso necessário esta opção pode ser alterada.

Caso o usuário **não** tenha interesse em **avaliar a incerteza dos parâmetros**, apenas **não** executar **Estime.incertezaParametros**.

In [15]:
Estimation.incertezaParametros(metodoIncerteza='Geral',preencherregiao=True)

## Predição e análise de resíduos

 No método predição, é feita a avaliação da grandeza dependente com base nos parâmetros avaliados, no modelo e nos dados das grandezas dependentes. Caso tenham dados para predição (no método setConjunto foi definido o tipo "predicao"), estes são usados. Além disso, este método avalia a matriz de covariância da predição, caso o método incertezaParametros tenha sido executado.
 
 Em análise de resíduos, é possível verificar alguns indicares sobre os resíduos da estimação, entre eles: (i) avaliação da média do resíduo, (ii) homocedasticidade dos resíduos (possíveis relações de dependencia e/ou tendencia entre as variaveis), (iii)  normalidade, (iv) $R^2$, (v) autocorrelação. Além de avaliação sobre o valor ótimo da função objetivo.
 
 A analise de residuos é feita prioritariamente com os dados de validação.==============================================================

In [16]:
Estimation.predicao()
Estimation.analiseResiduos()

## Gráficos e relatório

Nesta etapa ocorre a geração dos dados de saída do programa : relátorios e gráficos. 
Os gráficos são gerados de acordo com as etapas que foram realizadas. No relátorio contém informações a respeito.
dos testes estatisticos, função objetivo, matriz de covariância, status da otimização, dentre outros.

In [17]:
etapas = ['otimizacao', 'grandezas-entrada', 'predicao', 'grandezas-calculadas', 'analiseResiduos', 'regiaoAbrangencia']
Estimation.graficos(etapas)
Estimation.relatorio()

/home/giufba-admin/Documentos/MT_PEU/MT_PEU.py:2019: UserWarning: Os gráficos de otimizacao não puderam ser criados, o algoritmo de otimização utilizado não possui gráficos de desempenho OU o método otimizacao não foi executado.
  warn('Os gráficos de otimizacao não puderam ser criados, o algoritmo de otimização utilizado não possui gráficos de desempenho OU o método otimizacao não foi executado.',UserWarning)


<Figure size 432x288 with 0 Axes>

<Figure size 360x240 with 0 Axes>

Referências: 

[1] SANJARI, Ehsan. A new simple method for accurate calculation of saturated vapor pressure. Elsevier. 
p. 12-16. mar. 2013.

[2] ZAITSEVA, Ksenia V.; ZAITSAU, Dzmitry H.; VARFOLOMEEV, Mikhail A.. 
Vapour pressures and enthalpies of vaporisation of alkyl formamides. Elsevier. Alemanha, p. 228-238. maio 2019.

INMETRO.: Avaliação de dados de medição — Guia para a expressão de incerteza de medição. Rio de Janeiro: Jcgm, 2008.